In [1]:
# 기본적인 라이브러리(기초세션에서 다룸)
import os
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터

사용할 데이터 : bike.csv

주어진 데이터는 서울시 공공데이터인 자전거 대여 수를 나타낸 데이터로 아래와 같다. 데이터에는 자전거 대여 수와 함께 ID, 시간, 날씨 등의 정보를 포함하고 있다.

서울시 마포구의 날짜별, 시간별 기상상황과 따릉이 대여 수 데이터
즉, 따릉이 보관소별로 매일 자전거 대여

* id : 마포구에 있는 따릉이 보관소의 고유 id
* hour : 따릉이 보관소에서 기상상황을 측정한 시간
* temperature : 기온
* precipitation : 비가 오지 않았으면 0, 비가 오면 1, null은 비가 온 것도 아니고 안 온 것도 아니라서 센서가 확실히 측정불가한 상태
* windspeed : 풍속(평균)
* humidity : 습도
* visibility : 시정(視程), 시계(視界)(특정 기상 상태에 따른 가시성을 의미)
* ozone : 오존
* pm10 : 미세먼지(머리카락 굵기의 1/5에서 1/7 크기의 미세먼지)
* pm2.5 : 미세먼지(머리카락 굵기의 1/20에서 1/30 크기의 미세먼지)
* count : 측정한 날짜의 따릉이 대여 수

> 데이터 출처 : https://www.dacon.io/competitions/open/235576/data

# 모델 정리

모델별 라이브러리 정리(Classifier & Regressor)

각 모델은 회귀와 분류에 대한 라이브러리를 동시에 제공한다. 세션시간에 다룬 내용은 각 모델의 핵심원리에 대해 설명하기 위해 회귀나 분류 중에 더 설명하기 용이한 것을 선택하여 다루었다.

예를 들어, Voting의 Regressor은 세션시간에 Hard와 Soft Voting으로 설명하여 분류모델로 생각할 수 있지만, Regressor도 가능하는 것을 알려주는 라이브러리이다.

* DecisionTree(의사결정나무)
    * DecisionTree : DecisionTreeClassifier
    * RegressionTree(회귀나무) : DecisionTreeRegressor
* Ensemble(앙상블)
    * Voting : VotingClassifier & VotingRegressor
        * VotingClassifier 참고자료
          https://yganalyst.github.io/ml/ML_chap6-1/

        * VotingRegressor 참고자료
          1. https://runebook.dev/ko/docs/scikit_learn/modules/generated/sklearn.ensemble.votingregressor
          2. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingRegressor.html#sklearn.ensemble.VotingRegressor
    * Bagging : RandomForestClassifier & RandomForestRegressor
    * Boosting
        * GBM : GradientBoostingClassifier & GradientBoostingRegressor
        * XGBoost : XGBClassifier & XGBRegressor
        * LightGBM : LGBClassifier & LGBRegressor
        * CatBoost : CatBoostClassifier & CatBoostRegressor
            * CatBoost 참고자료 : https://catboost.ai/en/docs/concepts/python-reference_catboostclassifier
    * Stacking : https://lsjsj92.tistory.com/558

  
참고로 XGBoost, LightGBM, CatBoost은 sklearn이 아닌 다른 라이브러리를 사용한다.

# 중요 사항

이전 '[0131]_LinearRegression_and_SVM_HW'에서 다뤘던 모델인 [SVM, 회귀, 로지스틱회귀, Ridge&Lasso]의 경우에는 스케일링이 필요했다.

그러나 이번 시간의 '[0202]_DecisionTree_and_Ensemble'에 다루는 모델인 [DecisionTree, Ensemble]의 경우에는 스케일링이 필요없다.

> 반드시 해당 글을 끝까지 읽어주세요.
> https://syj9700.tistory.com/56

# 예제 1 : Regression 문제

이전 '[0131]_LinearRegression_and_SVM_HW' 예제의 bike.csv에서 y값을 count(연속형)으로 하는 regression문제를 해결해보자.


서울시 마포구의 날짜별, 시간별 기상상황과 따릉이 대여 수 데이터
즉, 따릉이 보관소별로 매일 자전거 대여

* id : 마포구에 있는 따릉이 보관소의 고유 id
* hour : 따릉이 보관소에서 기상상황을 측정한 시간
* temperature : 기온
* precipitation : 비가 오지 않았으면 0, 비가 오면 1, null은 비가 온 것도 아니고 안 온 것도 아니라서 센서가 확실히 측정불가한 상태
* windspeed : 풍속(평균)
* humidity : 습도
* visibility : 시정(視程), 시계(視界)(특정 기상 상태에 따른 가시성을 의미)
* ozone : 오존
* pm10 : 미세먼지(머리카락 굵기의 1/5에서 1/7 크기의 미세먼지)
* pm2.5 : 미세먼지(머리카락 굵기의 1/20에서 1/30 크기의 미세먼지)
* count : 측정한 날짜의 따릉이 대여 수

> 데이터 출처 : https://www.dacon.io/competitions/open/235576/data

## 데이터 전처리

In [2]:
# 데이터 불러오기

path = '/Users/jb/Downloads/[0131][HW]_LinearRegression_and_SVM/data/bike.csv'
bike = pd.read_csv(path)
bike.head()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431


In [3]:
# 결측치 처리 -> null이 포함된 행(데이터)은 삭제

bike.dropna(axis=0, inplace=True)
bike.isnull().sum()

id                        0
hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_humidity         0
hour_bef_visibility       0
hour_bef_ozone            0
hour_bef_pm10             0
hour_bef_pm2.5            0
count                     0
dtype: int64

In [4]:
# 범주형 X 데이터 one-hot encoding

temp = pd.get_dummies(bike.loc[:,'hour_bef_precipitation'])
temp.columns = ['hbp1','hbp2'] # 2개의 class(binary-class)이다.
temp.head()

,hbp1,hbp2
0,0,1
1,1,0
2,1,0
3,1,0
4,1,0


In [5]:
# one-hot encoding한 '범주형 X 데이터'로 바꿔주기

bike = bike.drop(['hour_bef_precipitation'], axis=1)
bike = pd.concat([bike, temp], axis=1)
bike.head()

,id,hour,hour_bef_temperature,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count,hbp1,hbp2
0,3,20,16.3,1.5,89.0,576.0,0.027,76.0,33.0,49,0,1
1,6,13,20.1,1.4,48.0,916.0,0.042,73.0,40.0,159,1,0
2,7,6,13.9,0.7,79.0,1382.0,0.033,32.0,19.0,26,1,0
3,8,23,8.1,2.7,54.0,946.0,0.040,75.0,64.0,57,1,0
4,9,18,29.5,4.8,7.0,2000.0,0.057,27.0,11.0,431,1,0


In [6]:
# ‘y_label’, ‘X_features’ 만들기
# random_state = 1로 지정

y_label = bike.loc[:,'count']
X_features = bike.drop(['count'], axis=1)

In [7]:
# 다중공선성 고려 (1)

from statsmodels.stats.outliers_influence import variance_inflation_factor

# 값을 저장할 데이터프레임 변수
vif = pd.DataFrame()

# 첫번째 features축은 columns이름으로
vif["features"] = X_features.columns # 독립변수들의 이름들로 구성된 features열 추가

# 두번째 축은 VIF값을 계산하여 집어넣기
vif["VIF Factor"] = [variance_inflation_factor(X_features.values, i) for i in range(X_features.shape[1])] # 리스트내포

vif

,features,VIF Factor
0,id,1.009803
1,hour,1.415846
2,hour_bef_temperature,1.709341
3,hour_bef_windspeed,1.699543
4,hour_bef_humidity,3.058976
5,hour_bef_visibility,3.723676
6,hour_bef_ozone,1.766954
7,hour_bef_pm10,1.664517
8,hour_bef_pm2.5,2.123214
9,hbp1,174.277772


In [8]:
# vif가 가장 큰 hbp1열 제거

X2_features = X_features.drop('hbp1', axis=1)

In [9]:
# 다중공선성 고려 (2)
# 가장 vif가 큰 값을 제거하여 다시 vif확인

from statsmodels.stats.outliers_influence import variance_inflation_factor

# 값을 저장할 데이터프레임 변수
vif = pd.DataFrame()

# 첫번째 features축은 columns이름으로
vif["features"] = X2_features.columns # 독립변수들의 이름들로 구성된 features열 추가

# 두번째 축은 VIF값을 계산하여 집어넣기
vif["VIF Factor"] = [variance_inflation_factor(X2_features.values, i) for i in range(X2_features.shape[1])] # 리스트내포

vif

# 보통 vif값이 10을 넘으면, 제거하지만 여기서는 vif값이 10보다 압도적으로 크지 않고
# 열의 수가 얼마 되지 않기에 'hour_bef_temperature'을 남겨두기로 하자.
# 개인적인 판단임으로, 해당 열을 제거해도 된다.

,features,VIF Factor
0,id,4.027977
1,hour,5.860958
2,hour_bef_temperature,15.775746
3,hour_bef_windspeed,6.993526
4,hour_bef_humidity,5.835661
5,hour_bef_visibility,7.415170
6,hour_bef_ozone,8.852221
7,hour_bef_pm10,5.679270
8,hour_bef_pm2.5,8.222608
9,hbp2,1.155935


In [10]:
# 데이터 분할

from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X2_features, y_label, train_size=0.8,
                                                    test_size=0.2, random_state=1)
print(train_X.shape, test_X.shape, train_y.shape, test_y.shape)

(1056, 10) (265, 10) (1056,) (265,)


제일 위의 '중요사항'에 따라 이번시간에 다루는 모델을 사용할 때, 스케일링이 필요없다.

## 모델 학습

사용할 모델
* DecisionTreeRegressor : RegressionTree(회귀나무) <- (1)
* Bagging : RandomForestRegressor <- (2)
* Boosting
  * XGBoost : XGBRegressor <- (3)
  * LightGBM : LGBRegressor <- (4)


학습방법
* 모델을 하이퍼라미터 값을 변경해가면서 10-Fold를 통해 학습데이터 평가. 즉, 학습데이터를 학습과 검증데이터로 쪼개는 과정을 반복하여 학습시키는 방법
  * 참고 : https://huidea.tistory.com/30

* RegressionTree 하이퍼파라미터 후보 : criterion : “squared_error”, “friedman_mse”, “absolute_error”, “poisson” , max_depth=5,10,15,20,None
* Random Forest 하이퍼파라미터 후보 : max_depth=3,5,10 , n_estimators=100,200,300 , random_state=1

* XGBoost 하이퍼파라미터 후보 : max_depth=3,5,10 , n_estimators=100,200,300 , learning_rate = 0.001,0.01,0.1,1 , gamma = 0.5,1,2  , random_state=1 

* LightGBM 하이퍼파라미터 후보 : max_depth=3,5,10 , n_estimators=100,200,300 , learning_rate = 0.001,0.01,0.1,1 , random_state=1

* 하이퍼파라미터 선택을 위한 평가지표 : R2_score
  * 참고 : R2_score의 경우, 선형회귀가 아닌 다른 회귀에서도 사용할 수 있는 지표이다. 즉, MSE(r-MSE), MAE와 같이 서로 다른 종류의 모델간에 비교가 가능한 지표이다.



> 하이퍼파라미터의 종류가 많음으로, cross_val_score()보다 GridSearchCV()를 사용하자.
> * 해당 함수들에 대한 정보를 확인하기 위해 반드시 아래 문서 확인
>   * https://techblog-history-younghunjo1.tistory.com/100

> 각 모델의 하이퍼파라미터에 대한 정보는 sklearn문서를 참고
> * ex. (RegressionTree) https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

In [11]:
from sklearn.model_selection import GridSearchCV

참고사항

* 그냥 학습
  * 학습에 사용하는 데이터 = train_X, train_y
  * 테스트에 사용하는 데이터 = test_X, test_y
* k-fold 학습
  * 학습에 사용하는 데이터 = train_X, train_y
  * 테스트에 사용하는 데이터 = train_X, train_y
* k-fold를 사용하는 이유
  * 데이터의 수가 너무 적어, test_X, test_y를 만들기 어려운 경우
  * 하이퍼파라미터를 찾을 시에, test_X, test_y을 잘 예측하는 모델이 아닌 train_X, train_y을 잘 예측하는 모델로 하여, 과적합을 막기 위함

### 1. RegressionTree

In [14]:
from sklearn.tree import DecisionTreeRegressor

# 모델 객체 선언
reg_tree = DecisionTreeRegressor() # 회귀모델

# train 데이터에 대해서 학습진행
reg_tree.fit(train_X, train_y)

# HyperParameter 후보군; 딕셔너리 형태로 저장
parameters={'criterion':['squared_error', 'friedman_mse', 'absolute_error', 'poisson'], # 분류에서 gini & entropy를 사용한 것과 다르다.
            'max_depth':[5,10,15,20,None]}

# HyperParameter를 Tuning
model_rt=GridSearchCV(estimator=reg_tree, param_grid=parameters,
                     scoring='r2', cv=10, refit=True)
# cv=10; 교차검증을 위한 값으로, kfold(train/validation을 나누는 한 방법)를 사용할 수도 있다.
# refit=True -> 가장 성능이 좋은 '최적의 하이퍼파라미터'을 찾아 선택(튜닝)된 모델객체를 생성한다.

# refit=True(default); 최적의 하이퍼파라미터(criterion='friedman_mse', max_depth=5)를 넣어서 모델 객체를 생성한다.
# 즉, reg_tree=DecisionTreeRegressor(criterion='friedman_mse', max_depth=5)를 자동으로 수행해준다.

# 객체는 이전에 'refit=True'에 의해 생성되었기에, 따로 최적의 하이퍼파라미터로 모델객체 선언없이
# 바로 train 데이터에 대한 학습을 진행하면 된다.
model_rt.fit(train_X,train_y)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error', 'poisson'],
                         'max_depth': [5, 10, 15, 20, None]},
             scoring='r2')

In [15]:
model_rt.best_params_

{'criterion': 'poisson', 'max_depth': 5}

### 2. RandomForestRegressor

In [16]:
# 시간이 조금 소요된다.

from sklearn.ensemble import RandomForestRegressor

# 객체 선언
randomforest=RandomForestRegressor() # 회귀모델

# train 데이터에 대해서 학습진행
randomforest.fit(train_X, train_y)

# HyperParameter 후보군
parameters={'max_depth':[3,5,10],
            'n_estimators':[100,200,300],
            'random_state':[1]} # RandomForest모델에서 랜덤추출이라는 과정이 이뤄짐으로 'random_state'를 Hyper-Parameter로 받는다.

# HyperParameter를 Tuning
model_rfr=GridSearchCV(estimator=randomforest, param_grid=parameters,
                 scoring='r2', cv=10, refit=True)

model_rfr.fit(train_X,train_y)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [3, 5, 10],
                         'n_estimators': [100, 200, 300], 'random_state': [1]},
             scoring='r2')

In [17]:
model_rfr.best_params_

{'max_depth': 10, 'n_estimators': 300, 'random_state': 1}

### 3. XGBRegressor

In [21]:
# 시간이 꽤 많이 소요된다.

from xgboost import XGBRegressor # sklearn 라이브러리에서 가져오는게 아니다.

# 객체 선언
xgboost=XGBRegressor(objective ='reg:squarederror') # 회귀모델
# objective ='reg:squarederror'; XGBRegressor라이브러리 버전관련 문제

# train 데이터에 대해서 학습진행
xgboost.fit(train_X, train_y)

# HyperParameter 후보군
parameters={'max_depth':[3,5,10],
            'n_estimators':[100,200,300],
            'learning_rate':[1e-3,0.01,0.1,1],
            'gamma':[0.5,1,2],
            'random_state':[1]}
# learning_rate의 경우에 model의 개개인적인 특징보다는 학습시에 적용되는 값이다.
# 따라서 xgboost=XGBRegressor(learning_rate=0.01) <- 오류 발생

# HyperParameter를 Tuning
model_xgb=GridSearchCV(estimator=xgboost, param_grid=parameters,
                     scoring='r2', cv=10, refit=True)

model_xgb.fit(train_X,train_y)

GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None,...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             param_grid={'gamma': [0.5, 1, 2],
                         'learning_rate': [0.001, 0.01, 0.1, 1],
                         'max_depth': [3, 5, 10],
                         'n_estimators': [100, 200, 300], 'random_state': [1]},
             scoring='r2')

In [22]:
model_xgb.best_params_

{'gamma': 0.5,
 'learning_rate': 0.1,
 'max_depth': 5,
 'n_estimators': 300,
 'random_state': 1}

### 4. LGBMRegressor

In [23]:
# 시간이 조금 소요된다.

from lightgbm import LGBMRegressor # sklearn 라이브러리에서 가져오는게 아니다.

# 객체 선언
lightgbm=LGBMRegressor() # 회귀모델

# train 데이터에 대해서 학습진행
lightgbm.fit(train_X, train_y)

# HyperParameter 후보군
parameters={'max_depth':[3,5,10],
            'n_estimators':[100,200,300],
            'learning_rate':[1e-3,0.01,0.1,1],
            'random_state':[1]}

# HyperParameter를 Tuning
model_lgbm=GridSearchCV(estimator=lightgbm, param_grid=parameters,
                     scoring='r2', cv=10, refit=True)

# 객체는 이전에 'refit=True'에 의해 생성되었기에, 바로 train 데이터에 대한 학습을 진행하면 된다.
model_lgbm.fit(train_X,train_y)

GridSearchCV(cv=10, estimator=LGBMRegressor(),
             param_grid={'learning_rate': [0.001, 0.01, 0.1, 1],
                         'max_depth': [3, 5, 10],
                         'n_estimators': [100, 200, 300], 'random_state': [1]},
             scoring='r2')

In [24]:
model_lgbm.best_params_

{'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 100, 'random_state': 1}

## 모델 평가

학습데이터로 평가했을 때 가장 좋은 성능을 보인 하이퍼 파라미터값의 RegressionTree, RandomForest, XGBoost, LightGBM모델을 테스트 데이터로 평가
* 서로 다른 회귀모형을 비교할 때, 사용하는 평가지표 : MSE

In [25]:
from sklearn.metrics import mean_squared_error

앞선 GridSearchCV(refit=True)를 하여 최적의 hyper-parameter에 대한 모델객체를 생성하고, model.fit(train_X)로 학습도 시켰기에 그 결과인 (model_rt, model_rfr, model_xgb, model_lgbm)를 따로 '객체선언,학습' 할 필요없이 바로 써주면 된다.

##### 일단 Train 데이터에 대해서 확인


In [26]:
# 'Train데이터 전부'에 대해 각 모델별 MSE값 확인

print("RegressionTree model : {:.3f}".format(mean_squared_error(train_y, model_rt.predict(train_X))))
print("RandomForest model : {:.3f}".format(mean_squared_error(train_y, model_rfr.predict(train_X))))
print("XGBoost model : {:.3f}".format(mean_squared_error(train_y, model_xgb.predict(train_X))))
print("LightGBM model : {:.3f}".format(mean_squared_error(train_y, model_lgbm.predict(train_X))))

RegressionTree model : 1739.419
RandomForest model : 324.928
XGBoost model : 41.270
LightGBM model : 251.911


MSE가 낮을수록 좋은 모델임으로 다음순으로 모델 성능이 좋다.
1. XGBoost
2. LightGBM
3. RandomForest
4. RegressionTree

In [27]:
# GridSearchCV()를 진행하여 교차검증을 수행할 때, '일부의 Train데이터인 Validation데이터'에 각 모델별 r2 확인
# r2의 경우에도 MSE(r-MSE),MAE와 같이 서로 다른 회귀모델끼리 성능비교가 가능한 지표이다.

print("R square score for RegressionTree model : {:.3f}".format(model_rt.best_score_))
print("R square score for RandomForest model : {:.3f}".format(model_rfr.best_score_))
print("R square score for XGBoost model : {:.3f}".format(model_xgb.best_score_))
print("R square score for LightGBM model : {:.3f}".format(model_lgbm.best_score_))

R square score for RegressionTree model : 0.666
R square score for RandomForest model : 0.769
R square score for XGBoost model : 0.782
R square score for LightGBM model : 0.768


R square score가 클수록 좋은 모델임으로 다음순으로 모델 성능이 좋다.
1. XGBoost
2. RandomForest
3. LightGBM
4. RegressionTree

##### Test 데이터에 대해서 평가

In [28]:
# 'Test데이터'에 대해 각 모델별 MSE값 확인

print("RegressionTree model : {:.3f}".format(mean_squared_error(test_y, model_rt.predict(test_X))))
print("RandomForest model : {:.3f}".format(mean_squared_error(test_y, model_rfr.predict(test_X))))
print("XGBoost model : {:.3f}".format(mean_squared_error(test_y, model_xgb.predict(test_X))))
print("LightGBM model : {:.3f}".format(mean_squared_error(test_y, model_lgbm.predict(test_X))))

RegressionTree model : 2130.817
RandomForest model : 1730.452
XGBoost model : 1630.148
LightGBM model : 1791.900


MSE가 낮을수록 좋은 모델임으로 다음순으로 모델 성능이 좋다.
1. XGBoost
2. RandomForest
3. LightGBM
4. RegressionTree

# 문제 1 : Classification 문제

해당 코드의 빈 곳(??)을 찾아 완성해주세요.

이전 '[0131]_LinearRegression_and_SVM_HW' 예제의 bike.csv에서 y값을 precipitation(범주형)으로 하는 classification문제를 해결해보자.


서울시 마포구의 날짜별, 시간별 기상상황과 따릉이 대여 수 데이터
즉, 따릉이 보관소별로 매일 자전거 대여

* id : 마포구에 있는 따릉이 보관소의 고유 id
* hour : 따릉이 보관소에서 기상상황을 측정한 시간
* temperature : 기온
* precipitation : 비가 오지 않았으면 0, 비가 오면 1, null은 비가 온 것도 아니고 안 온 것도 아니라서 센서가 확실히 측정불가한 상태
* windspeed : 풍속(평균)
* humidity : 습도
* visibility : 시정(視程), 시계(視界)(특정 기상 상태에 따른 가시성을 의미)
* ozone : 오존
* pm10 : 미세먼지(머리카락 굵기의 1/5에서 1/7 크기의 미세먼지)
* pm2.5 : 미세먼지(머리카락 굵기의 1/20에서 1/30 크기의 미세먼지)
* count : 측정한 날짜의 따릉이 대여 수

> 데이터 출처 : https://www.dacon.io/competitions/open/235576/data

## 데이터 전처리

In [29]:
# 데이터 불러오기

path = '/Users/jb/Downloads/[0131][HW]_LinearRegression_and_SVM/data/bike.csv'
bike = pd.read_csv(path)
bike.head()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431


In [32]:
# 결측치 처리 -> null이 포함된 행(데이터)은 삭제

bike.dropna(inplace = True)
bike.isnull().sum()

id                        0
hour                      0
hour_bef_temperature      0
hour_bef_precipitation    0
hour_bef_windspeed        0
hour_bef_humidity         0
hour_bef_visibility       0
hour_bef_ozone            0
hour_bef_pm10             0
hour_bef_pm2.5            0
count                     0
dtype: int64

In [27]:
# 범주형 X 데이터 one-hot encoding
# 유일하게 범주형 데이터였던, 'hour_bef_precipitation'열이 y값이 되었음으로
# 남은 X 열들(features)은 모두 연속형 데이터임으로, 따로 'one-hot encoding'을 해줄 필요가 없다.

In [39]:
# 'y_label', 'X_features' 만들기

y_label = bike.loc[:,'hour_bef_precipitation']
X_features = bike.drop(['id','hour_bef_precipitation'], axis=1)

In [40]:
# 다중공선성 고려 (1)

from statsmodels.stats.outliers_influence import variance_inflation_factor

# 값을 저장할 데이터프레임 변수
vif = pd.DataFrame()

# 첫번째 features축은 columns이름으로
vif["features"] = X_features.columns # 독립변수들의 이름들로 구성된 features열 추가

# 두번째 축은 VIF값을 계산하여 집어넣기
vif["VIF Factor"] = [variance_inflation_factor(X_features.values, i) for i in range(X_features.shape[1])] # 리스트내포

vif

,features,VIF Factor
0,hour,7.231942
1,hour_bef_temperature,18.415036
2,hour_bef_windspeed,6.963226
3,hour_bef_humidity,5.448072
4,hour_bef_visibility,6.807508
5,hour_bef_ozone,8.834388
6,hour_bef_pm10,5.852081
7,hour_bef_pm2.5,8.101729
8,count,6.614504


In [41]:
# vif가 가장 큰 ??열 제거

X2_features = X_features.drop('hour_bef_temperature', axis=1)

In [42]:
# 다중공선성 고려 (2)
# 가장 vif가 큰 값을 제거하여 다시 vif확인

from statsmodels.stats.outliers_influence import variance_inflation_factor

# 값을 저장할 데이터프레임 변수
vif = pd.DataFrame()

# 첫번째 features축은 columns이름으로
vif["features"] = X2_features.columns # 독립변수들의 이름들로 구성된 features열 추가

# 두번째 축은 VIF값을 계산하여 집어넣기
vif["VIF Factor"] = [variance_inflation_factor(X2_features.values, i) for i in range(X2_features.shape[1])] # 리스트내포

vif

# vif값이 10을 넘지 않음으로, 다중공선성을 걱정하여 열을 제거할 필요가 없다.

,features,VIF Factor
0,hour,7.231647
1,hour_bef_windspeed,6.957576
2,hour_bef_humidity,5.177573
3,hour_bef_visibility,5.526070
4,hour_bef_ozone,7.760598
5,hour_bef_pm10,5.759442
6,hour_bef_pm2.5,7.827325
7,count,5.654690


In [43]:
# X, y 를 8: 2로 데이터 분할
# stratify를 통해 y값 클래스 비율 일정하게 분할; 분류일 경우, 데이터분할시에 주의가 필요하다.
# random_state = 1로 지정

from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X2_features, y_label, train_size=0.8,
                                                    test_size=0.2, random_state=1, stratify=y_label)
print(train_X.shape, test_X.shape, train_y.shape, test_y.shape)

(1056, 8) (265, 8) (1056,) (265,)


제일 위의 '중요사항'에 따라 이번시간에 다루는 모델을 사용할 때, 스케일링이 필요없다.

## 모델 학습

사용할 모델
* DecisionTreeClassifier : DecisionTree(의사결정나무) <- (1)
* Bagging : RandomForestClassifier <- (2)
* Boosting
  * XGBoost : XGBClassifier <- (3)
  * LightGBM : LGBClassifier <- (4)


학습방법
* 모델을 하이퍼라미터 값을 변경해가면서 10-Fold를 통해 학습데이터 평가. 즉, 학습데이터를 학습과 검증데이터로 쪼개는 과정을 반복하여 학습시키는 방법
  * 참고 : https://huidea.tistory.com/30

* DecisionTree 하이퍼파라미터 후보 : criterion : 'gini', 'entropy' , max_depth=5,10,15,20,None
* Random Forest 하이퍼파라미터 후보 : max_depth=3,5,10 , n_estimators=100,200,300 , random_state=1

* XGBoost 하이퍼파라미터 후보 : max_depth=3,5,10 , n_estimators=100,200,300 , learning_rate = 0.001,0.01,0.1,1 , gamma = 0.5,1,2  , random_state=1 

* LightGBM 하이퍼파라미터 후보 : max_depth=3,5,10 , n_estimators=100,200,300 , learning_rate = 0.001,0.01,0.1,1 , random_state=1

* 하이퍼파라미터 선택을 위한 평가지표 : F1-Score



> 하이퍼파라미터의 종류가 많음으로, cross_val_score()보다 GridSearchCV()를 사용하자.
> * 해당 함수들에 대한 정보를 확인하기 위해 반드시 아래 문서 확인
>   * https://techblog-history-younghunjo1.tistory.com/100

> 각 모델의 하이퍼파라미터에 대한 정보는 sklearn문서를 참고
> * ex. (DecisionTree) https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier

In [44]:
from sklearn.model_selection import GridSearchCV

참고사항

* 그냥 학습
  * 학습에 사용하는 데이터 = train_X, train_y
  * 테스트에 사용하는 데이터 = test_X, test_y
* k-fold 학습
  * 학습에 사용하는 데이터 = train_X, train_y
  * 테스트에 사용하는 데이터 = train_X, train_y
* k-fold를 사용하는 이유
  * 데이터의 수가 너무 적어, test_X, test_y를 만들기 어려운 경우
  * 하이퍼파라미터를 찾을 시에, test_X, test_y을 잘 예측하는 모델이 아닌 train_X, train_y을 잘 예측하는 모델로 하여, 과적합을 막기 위함

### 1. DecisionTree

In [45]:
from sklearn.tree import DecisionTreeClassifier

# 모델 객체 선언
clf_tree = DecisionTreeClassifier() # 분류모델

# train 데이터에 대해서 학습진행
clf_tree.fit(train_X, train_y)

# HyperParameter 후보군; 딕셔너리 형태로 저장
parameters={'criterion':['gini', 'entropy'],
            'max_depth':[5,10,15,20,None]}

# HyperParameter를 Tuning
model_dt=GridSearchCV(estimator=clf_tree, param_grid=parameters,
                     scoring='f1', cv=10, refit=True)

model_dt.fit(train_X,train_y)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 10, 15, 20, None]},
             scoring='f1')

In [46]:
model_dt.best_params_

{'criterion': 'gini', 'max_depth': 10}

### 2. RandomForestClassifier

In [48]:
from sklearn.ensemble import RandomForestClassifier

# 객체 선언
randomforest=RandomForestClassifier() # 분류모델

# train 데이터에 대해서 학습진행
randomforest.fit(train_X, train_y)

# HyperParameter 후보군
parameters = {'max_depth' : [3, 5, 10],
              'n_estimators' : [100, 200, 300],
              'random_state' : [1]}

# HyperParameter를 Tuning
model_rfc=GridSearchCV(estimator=randomforest, param_grid=parameters,
                       scoring='f1', cv=10, refit=True)

model_rfc.fit(train_X,train_y)

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 5, 10],
                         'n_estimators': [100, 200, 300], 'random_state': [1]},
             scoring='f1')

In [49]:
model_rfc.best_params_

{'max_depth': 10, 'n_estimators': 300, 'random_state': 1}

### 3. XGBClassifier

In [61]:
from xgboost import XGBClassifier # sklearn 라이브러리에서 가져오는게 아니다.

# 객체 선언
xgboost=XGBClassifier() # 분류모델

xgboost.fit(train_X, train_y)

parameters = {'max_depth' : [3, 5, 10],
              'n_estimators' : [100, 200, 300],
              'learning_rate' : [0.001, 0.01, 0.1, 1],
              'gamma' : [0.5, 1, 2],
              'random_state' : [1]}

model_xgb=GridSearchCV(estimator=xgboost, param_grid=parameters,
                       scoring='f1', cv=10, refit=True)

model_xgb.fit(train_X,train_y)

GridSearchCV(cv=10,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             param_grid={'gamma': [0.5, 1, 2],
                         'learning_rate': [0.001, 0.01, 0.1, 1],
                         'max_depth': [3, 5, 10],
                         'n_estimators': [100, 200, 300], 'random_state': [1]},
             scoring='f1')

In [62]:
model_xgb.best_params_

{'gamma': 1,
 'learning_rate': 0.1,
 'max_depth': 10,
 'n_estimators': 100,
 'random_state': 1}

### 4. LGBMClassifier

In [63]:
from lightgbm import LGBMClassifier # sklearn 라이브러리에서 가져오는게 아니다.

# 객체 선언
lightgbm=LGBMClassifier() # 회귀모델

lightgbm.fit(train_X, train_y)

parameters = {'max_depth' : [3, 5, 10],
              'n_estimators' : [100, 200, 300],
              'learning_rate' : [0.001, 0.01, 0.1, 1],
              'random_state' : [1]}

model_lgbm=GridSearchCV(estimator=lightgbm, param_grid=parameters,
                       scoring='f1', cv=10, refit=True)

model_lgbm.fit(train_X,train_y)

GridSearchCV(cv=10, estimator=LGBMClassifier(),
             param_grid={'learning_rate': [0.001, 0.01, 0.1, 1],
                         'max_depth': [3, 5, 10],
                         'n_estimators': [100, 200, 300], 'random_state': [1]},
             scoring='f1')

In [64]:
model_lgbm.best_params_

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'random_state': 1}

## 모델 평가

학습데이터로 평가했을 때 가장 좋은 성능을 보인 하이퍼 파라미터값의 DecisionTree, RandomForest, XGBoost, LightGBM모델을 테스트 데이터로 평가
* 서로 다른 분류모형을 비교할 때, 사용하는 평가지표 : Accuracy, Recall, Precision ,F1-Score

In [65]:
from sklearn.metrics import accuracy_score,recall_score,precision_score, f1_score

앞선 GridSearchCV(refit=True)를 하여 최적의 hyper-parameter에 대한 모델객체를 생성하고, model.fit(train_X)로 학습도 시켰기에 그 결과인 (model_dt, model_rfc, model_xgb, model_lgbm)를 따로 '객체선언,학습' 할 필요없이 바로 써주면 된다.

##### 일단 Train 데이터에 대해서 확인



In [66]:
# 'Train데이터 전부'에 대해 각 모델별 f1-score값 확인

print("DecisionTree model : {:.3f}".format(f1_score(train_y, model_dt.predict(train_X))))
print("RandomForest model : {:.3f}".format(f1_score(train_y, model_rfc.predict(train_X))))
print("XGBoost model : {:.3f}".format(f1_score(train_y, model_xgb.predict(train_X))))
print("LightGBM model : {:.3f}".format(f1_score(train_y, model_lgbm.predict(train_X))))

DecisionTree model : 1.000
RandomForest model : 1.000
XGBoost model : 0.933
LightGBM model : 1.000


F1-Score가 높을수록 좋은 모델임으로 다음순으로 모델 성능이 좋다.
1. LightGBM
2. DecisionTree
3. RandomForest
4. XGBoost

id도 뺐더니 결과가 다르게 나온듯

In [67]:
# GridSearchCV()를 진행하여 교차검증을 수행할 때, '일부의 Train데이터인 Validation데이터'에 각 모델별 f1-score값 확인
# f1-score의 경우에 서로 다른 분류모델끼리 성능비교가 가능한 지표이다.

print("DecisionTree model : {:.3f}".format(model_dt.best_score_))
print("RandomForest model : {:.3f}".format(model_rfc.best_score_))
print("XGBoost model : {:.3f}".format(model_xgb.best_score_))
print("LightGBM model : {:.3f}".format(model_lgbm.best_score_))

DecisionTree model : 0.493
RandomForest model : 0.458
XGBoost model : 0.514
LightGBM model : 0.458


F1-Score가 높을수록 좋은 모델임으로 다음순으로 모델 성능이 좋다.
1. XGBoost
2. DecisionTree
3. RandomForest, LightGBM

##### Test 데이터에 대해서 평가

In [68]:
# 'Test데이터'에 대해 각 모델별 accuracy_score값 확인

print("DecisionTree model : {:.3f}".format(accuracy_score(test_y, model_dt.predict(test_X))))
print("RandomForest model : {:.3f}".format(accuracy_score(test_y, model_rfc.predict(test_X))))
print("XGBoost model : {:.3f}".format(accuracy_score(test_y, model_xgb.predict(test_X))))
print("LightGBM model : {:.3f}".format(accuracy_score(test_y, model_lgbm.predict(test_X))))

DecisionTree model : 0.958
RandomForest model : 0.974
XGBoost model : 0.977
LightGBM model : 0.974


In [69]:
# 'Test데이터'에 대해 각 모델별 recall_score 확인

print("DecisionTree model : {:.3f}".format(recall_score(test_y, model_dt.predict(test_X))))
print("RandomForest model : {:.3f}".format(recall_score(test_y, model_rfc.predict(test_X))))
print("XGBoost model : {:.3f}".format(recall_score(test_y, model_xgb.predict(test_X))))
print("LightGBM model : {:.3f}".format(recall_score(test_y, model_lgbm.predict(test_X))))

DecisionTree model : 0.375
RandomForest model : 0.375
XGBoost model : 0.500
LightGBM model : 0.375


In [47]:
# 'Test데이터'에 대해 각 모델별 precision_score 확인

print("DecisionTree model : {:.3f}".format(precision_score(test_y, model_dt.predict(test_X))))
print("RandomForest model : {:.3f}".format(precision_score(test_y, model_rfc.predict(test_X))))
print("XGBoost model : {:.3f}".format(precision_score(test_y, model_xgb.predict(test_X))))
print("LightGBM model : {:.3f}".format(precision_score(test_y, model_lgbm.predict(test_X))))

DecisionTree model : 0.400
RandomForest model : 1.000
XGBoost model : 0.800
LightGBM model : 0.600


In [70]:
# 'Test데이터'에 대해 각 모델별 f1_score 확인

print("DecisionTree model : {:.3f}".format(f1_score(test_y, model_dt.predict(test_X))))
print("RandomForest model : {:.3f}".format(f1_score(test_y, model_rfc.predict(test_X))))
print("XGBoost model : {:.3f}".format(f1_score(test_y, model_xgb.predict(test_X))))
print("LightGBM model : {:.3f}".format(f1_score(test_y, model_lgbm.predict(test_X))))

DecisionTree model : 0.353
RandomForest model : 0.462
XGBoost model : 0.571
LightGBM model : 0.462


분류의 경우, 각 평가함수들은 목적에 따라 다르게 사용해야한다.

따라서 해당 문제의 경우, y값인 'hour_bef_precipitation'에서 비오는 것(1)과 비가 오지 않는 것(0) 중 어떤 것이 중요한지 생각하여 평가해야한다.

예를 들면,

Positive(1=비가옴)인 데이터를 Negative(0=비가안옴)로 잘못 판단하는 것이 큰 문제인 경우 재현율이 상대적으로 더 중요해지며, Negative(0=비가안옴)인 데이터를 Positive(1=비가옴)로 잘못 판단하는 것이 큰 문제인 경우 정밀도가더 중요해진다.

아래 글 참고
> https://ek-koh.github.io/data%20analysis/evaluation/

In [75]:
print(len(y_label[y_label == 1]))
print(len(y_label[y_label == 0]))

40
1281


위 평가지표와 글을 보고 모델을 비교하여 서술

위의 클래스 별 데이터 개수를 확인해보면 불균형하기 때문에 recall과 precision의 조화 평균인 f1 score를 사용하여 판단하는 것이 바람직하다. XGBoost 모델이 f1 score가 가장 높게 나타나고 있으므로 일반적인 성능이 뛰어나다고 말할 수 있다. 또한 recall도 XGBoost가 가장 높게 나타나므로 비가 오는 날에 비가 온다고 예측을 할 확률이 제일 높다. 

# 더 공부하고 싶으신 분들을 위한 자료


**Chapter 6 – Decision Trees**

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ageron/handson-ml2/blob/master/06_decision_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ageron/handson-ml2/blob/master/06_decision_trees.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

**Chapter 7 – Ensemble Learning and Random Forests**

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/ageron/handson-ml2/blob/master/07_ensemble_learning_and_random_forests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/ageron/handson-ml2/blob/master/07_ensemble_learning_and_random_forests.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" /></a>
  </td>
</table>

# 수고하셨습니다.